## Implementing binary decision trees

The goal of this notebook is to implement your own binary decision tree classifier. We will:
    
  - Use SFrames to do some feature engineering.
  - Transform categorical variables into binary variables.
  - Write a function to compute the number of misclassified examples in an intermediate node.
  - Write a function to find the best feature to split on.
  - Build a binary decision tree from scratch.
  - Make predictions using the decision tree.
  - Evaluate the accuracy of the decision tree.
  - Visualize the decision at the root node.

**Important Note**: In this assignment, we will focus on building decision trees where the data contain **only binary (0 or 1) features**. This allows us to avoid dealing with:
  -  Multiple intermediate nodes in a split
  - The thresholding issues of real-valued features.

This assignment **may be challenging**, so brace yourself :)

# Fire up Turi Create

Make sure you have the latest version of Turi Create.

In [ ]:
import turicreate

# Load the lending club dataset

We will be using the same [LendingClub](https://www.lendingclub.com/) dataset as in the previous assignment.

In [ ]:
loans = turicreate.SFrame('../data/lending-club-data.sframe/')

Like the previous assignment, we reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.

In [ ]:
## Add column safe_loans
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.remove_column('bad_loans')

Unlike the previous assignment where we used several features, in this assignment, we will just be using 4 categorical
features: 

1. grade of the loan 
2. the length of the loan term
3. the home ownership status: own, mortgage, rent
4. number of years of employment.

Since we are building a binary decision tree, we will have to convert these categorical features to a binary representation in a subsequent section using 1-hot encoding.

In [ ]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]
loans.column_names()

Let's explore what the dataset looks like.

In [ ]:
len(loans), loans.head(7)

## Subsample dataset to make sure classes are balanced

Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We use `seed=1` so everyone gets the same results.

In [ ]:
def downsample(loans):
  safe_loans_raw = loans[loans[target] == 1]
  risky_loans_raw = loans[loans[target] == -1]
  ## Since there are less risky loans than safe loans, find the ratio of the sizes
  ## and use that percentage to undersample the safe loans.
  perc = float(len(risky_loans_raw) / len(safe_loans_raw))
  safe_loans = safe_loans_raw.sample(perc, seed = 1)
  risky_loans = risky_loans_raw
  loans_data = risky_loans.append(safe_loans)
  n = len(loans_data)
  return (loans_data, safe_loans, risky_loans, float(len(safe_loans) / n), float(len(risky_loans) / n))

In [ ]:
loans_data, safe_loans, risky_loans, p_safe, p_risky = downsample(loans)

print(f"Percentage of safe loans                 :{p_safe:1.3f}")
print(f"Percentage of risky loans                :{p_risky:1.3f}")
print(f"Total number of loans in our new dataset :{len(loans_data)}")

**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in "[Learning from Imbalanced Data](http://www.ele.uri.edu/faculty/he/PDFfiles/ImbalancedLearning.pdf)" by Haibo He and Edwardo A. Garcia, *IEEE Transactions on Knowledge and Data Engineering* **21**(9) (June 26, 2009), p. 1263–1284. For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

## Transform categorical data into binary features

In this assignment, we will implement **binary decision trees** (decision trees for binary features, a specific case of categorical variables taking on two values, e.g., true/false). Since all of our features are currently categorical features, we want to turn them into binary features. 

For instance, the `home_ownership` feature represents the home ownership status of the loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

Since this code requires a few Python and Turi Create tricks, feel free to use this block of code as is. Refer to the API documentation for a deeper understanding.

In [ ]:
def hot_encode(loans_data, features):
  for f in features:
    loans_data_ohe = loans_data[f].apply(lambda x: {x: 1})  ## ohe == one_hot_encoded
    loans_data_unp = loans_data_ohe.unpack(column_name_prefix=f)
    ## Change None's to 0's
    for col in loans_data_unp.column_names():
        loans_data_unp[col] = loans_data_unp[col].fillna(0)
    loans_data = loans_data.remove_column(f)
    loans_data = loans_data.add_columns(loans_data_unp)
  return loans_data

In [ ]:
loans_data = risky_loans.append(safe_loans)
loans_data[1:5]

In [ ]:
loans_data = hot_encode(loans_data, features)
loans_data[0:5]

Let's see what the feature columns look like now:

In [ ]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable
features

In [ ]:
## Number of features
len(features)

Let's explore what one of these columns looks like:

In [ ]:
loans_data['grade.A']

This column is set to 1 if the loan grade is A and 0 otherwise.

**Checkpoint:** Make sure the following answers match up.

In [ ]:
assert loans_data['grade.A'].sum() == 6422

## Train-test split

We split the data into a train test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [ ]:
train_data, test_data = loans_data.random_split(.8, seed=1)
train_data.shape, test_data.shape

# Decision tree implementation

In this section, we will implement binary decision trees from scratch. There are several steps involved in building a decision tree. For that reason, we have split the entire assignment into several sections.

## Function to count number of mistakes while predicting majority class

Recall from the lecture that prediction at an intermediate node works by predicting the **majority class** for all data points that belong to this node.

Now, we will write a function that calculates the number of **missclassified examples** when predicting the **majority class**. This will be used to help determine which feature is the best to split on at a given node of the tree.

**Note**: Keep in mind that in order to compute the number of mistakes for a majority classifier, we only need the label (y values) of the data points in the node. 

**Steps to follow**:
  1. Calculate the number of safe loans and risky loans.
  1. Since we are assuming majority class prediction, all the data points that are **not** in the majority class are considered **mistakes**.
  1. Return the number of **mistakes**.


Now, let us write the function `inter_node_num_mistakes` which computes the number of misclassified examples of an intermediate node given the set of labels (y values) of the data points contained in the node.

In [ ]:
def inter_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0: return 0
    n_pos = (labels_in_node == 1).sum()        ## Count the number of 1's (safe loans)
    n_neg = len(labels_in_node)  - n_pos       ## (labels_in_node == -1).sum()  ## Count the number of -1's (risky loans)
    return n_neg if n_pos >= n_neg else n_pos  ## Return the number of mistakes that the majority classifier makes.


Because there are several steps in this assignment, we have introduced some stopping points where you can check your code and make sure it is correct before proceeding. <br />
To test your `inter_node_num_mistakes` function, run the following code until you get a **Test passed!**, then you should proceed. Otherwise, you should spend some time figuring out where things went wrong.

In [ ]:
## Test case 1
example_labels = turicreate.SArray([-1, -1, 1, 1, 1])
assert inter_node_num_mistakes(example_labels) == 2, 'Test 1 failed... try again!'

## Test case 2
example_labels = turicreate.SArray([-1, -1, 1, 1, -1, 1, 1])
assert inter_node_num_mistakes(example_labels) == 3, 'Test 2 failed... try again!'

## Test case 3
example_labels = turicreate.SArray([-1, -1, -1, -1, 1, -1, 1])
assert inter_node_num_mistakes(example_labels) == 2, 'Test 3 failed... try again!'

## Function to pick best feature to split on

The function **best_splitting_feature** takes 3 arguments: 
1. The data (SFrame of data which includes all of the feature columns and label column)
2. The features to consider for splits (a list of strings of column names to consider for splits)
3. The name of the target/label column (string)

The function will loop through the list of possible features, and consider splitting on each of them. It will calculate the classification error of each split and return the feature that had the smallest classification error when split on.

Recall that the **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

Follow these steps: 
* **Step 1:** Loop over each feature in the feature list
* **Step 2:** Within the loop, split the data into two groups: one group where all of the data has feature value 0 or False (we will call this the **left** split), and one group where all of the data has feature value 1 or True (we will call this the **right** split). Make sure the **left** split corresponds with 0 and the **right** split corresponds with 1 to ensure your implementation fits with our implementation of the tree building process.
* **Step 3:** Calculate the number of misclassified examples in both groups of data and use the above formula to compute the **classification error**.
* **Step 4:** If the computed error is smaller than the best error found so far, store this **feature and its error**.

This may seem like a lot, but we have provided pseudocode in the comments in order to help you implement the function correctly.

**Note:** Remember that since we are only dealing with binary features, we do not have to consider thresholds for real-valued features. This makes the implementation of this function much easier.

Fill in the places where you find `## YOUR CODE HERE`. There are **five** places in this function for you to fill in.

In [ ]:
def best_splitting_feature(data, features, target):
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should initialize it with something larger than 1.
    
    num_data_points = len(data)  
    for feature in features:    
        left_split = data[data[feature] == 0]  ## The left split will have all data points where the feature value is 0 => SFrame
        right_split = data[data[feature] == 1] ## The right split will have all data points where the feature value is 1
        
        left_mistakes = inter_node_num_mistakes(left_split[target])   ## Calc. number of misclassified examples in the left split. (SArray)   
        right_mistakes = inter_node_num_mistakes(right_split[target]) ## Calc. number of misclassified examples in the right split.
            
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes + right_mistakes) / num_data_points # Compute the classification error of this split.

        if error < best_error:  ## keep the feature as best_feature and the error as best_error
          best_error = error
          best_feature = feature

    return best_feature # Return the best feature we found

To test your `best_splitting_feature` function, run the following code:

In [ ]:
assert best_splitting_feature(train_data, features, 'safe_loans') == 'term. 36 months', 'Test failed... try again!'

## Building the tree

With the above functions implemented correctly, we are now ready to build our decision tree. Each node in the decision tree is represented as a dictionary which contains the following keys and possible values:

    { 
       'is_leaf'            : True/False.
       'prediction'         : Prediction at the leaf node.
       'left'               : (dictionary corresponding to the left tree).
       'right'              : (dictionary corresponding to the right tree).
       'splitting_feature'  : The feature that this node splits on.
    }

First, we will write a function that creates a leaf node given a set of target values. 

In [ ]:
def create_leaf(target_values):
    ## Create a leaf node
    leaf = {
      'splitting_feature' : None,
      'left' : None,
      'right' : None,
      'is_leaf': True,   ## YOUR CODE HERE 
    }  
    
    ## Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    ## For the leaf node, set the prediction to be the majority class.
    ## Store the predicted class (1 or -1) in leaf['prediction']
    leaf['prediction'] = 1 if num_ones > num_minus_ones else -1
    return leaf 

We have provided a function that learns the decision tree recursively and implements 3 stopping conditions:
1. **Stopping condition 1:** All data points in a node are from the same class.
2. **Stopping condition 2:** No more features to split on.
3. **Additional stopping condition:** In addition to the above two stopping conditions covered in lecture, in this assignment we will also consider a stopping condition based on the **max_depth** of the tree. By not letting the tree grow too deep, we will save computational effort in the learning process. 

Now, we will write down the skeleton of the learning algorithm. Fill in the places where you find `## YOUR CODE HERE`. There are **seven** places in this function for you to fill in.

In [ ]:
def decision_tree_create(data, features, target, current_depth=0, max_depth=10, verbose=False):
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    if verbose:
      print("--------------------------------------------------------------------")
      print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))

    # Stopping condition 1
    # (Check if there are mistakes at current node, using inter_node_num_mistakes)
    if inter_node_num_mistakes(target_values) == 0:
        if verbose: print("Stopping condition 1 reached.")
        return create_leaf(target_values)    ## If not mistakes at current node, make current node a leaf node
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if len(remaining_features) == 0: 
        if verbose: print("Stopping condition 2 reached.")
        return create_leaf(target_values)    ## If there are no remaining features to consider, make current node a leaf node
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth: 
        if verbose: print("Reached maximum depth. Stopping for now.")
        return create_leaf(target_values)    ## If the max tree depth has been reached, make current node a leaf node

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    remaining_features.remove(splitting_feature)

    if verbose:
      print("Split on feature %s. (%s, %s)" % (splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        if verbose: print("Creating leaf node.")
        return create_leaf(left_split[target])
      
    if len(right_split) == len(data):
        if verbose: print("Creating leaf node.")
        return create_leaf(right_split[target])

    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth, verbose)
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth, verbose)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

Here is a recursive function to count the nodes in your tree:

In [ ]:
def count_nodes(tree):
    return 1 if tree is None or tree['is_leaf'] else 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

Run the following test code to check your implementation. Make sure you get **'Test passed'** before proceeding.

In [ ]:
small_data_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth=3, verbose=True)

if count_nodes(small_data_decision_tree) == 13:
    print('Test passed!')
else:
    print('Test failed... try again!')
    print('Number of nodes found                :', count_nodes(small_data_decision_tree))
    print('Number of nodes that should be there : 13' )

## Build the tree!

Now that all the tests are passing, we will train a tree model on the **train_data**. Limit the depth to 6 (**max_depth = 6**) to make sure the algorithm doesn't run for too long. Call this tree `dt6_model`. 

**Warning**: This code block may take 1-2 minutes to learn. 

In [ ]:
# Make sure to cap the depth at 6 by using max_depth = 6
dt6_model = decision_tree_create(train_data, features, 'safe_loans', max_depth=6, verbose=False)

## Making predictions with a decision tree

As discussed in the lecture, we can make predictions from the decision tree with a simple recursive function. Below, we call this function `classify`, which takes in a learned `tree` and a test point `x` to classify.  We include an option `annotate` that describes the prediction path when set to `True`.

In [ ]:
def classify(tree, x, annotate=False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
      if annotate: 
        print("At leaf, predicting %s" % tree['prediction'])
      return tree['prediction'] 
    else:
      # split on feature.
      split_feature_value = x[tree['splitting_feature']]
      if annotate: 
        print("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
      return classify(tree['left'], x, annotate) if split_feature_value == 0 else classify(tree['right'], x, annotate)    

Now, let's consider the first example of the test set and see what `dt6_model` predicts for this data point.

In [ ]:
test_data[0]

In [ ]:
print('Predicted class: %s ' % classify(dt6_model, test_data[0]))

Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [ ]:
classify(dt6_model, test_data[0], annotate=True)

**Quiz Question:** What was the feature that `dt6_model` first split on while making the prediction for test_data[0]?
  - 'term. 36 months'

**Quiz Question:** What was the first feature that lead to a right split of test_data[0]?
 - `grade.D` (as split_feature_value was 1)

**Quiz Question:** What was the last feature split on before reaching a leaf node for test_data[0]?
 - `grade.D`

## Evaluating your decision tree

Now, we will write a function to evaluate a decision tree by computing the classification error of the tree on the given dataset.

Again, recall that the **classification error** is defined as follows:
$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

Now, write a function called `evaluate_classification_error` that takes in as input:
1. `tree` (as described above)
2. `data` (an SFrame)
3. `target` (a string - the name of the target/label column)

This function should calculate a prediction (class label) for each row in `data` using the decision `tree` and return the classification error computed using the above formula. Fill in the places where you find `## YOUR CODE HERE`. There is **one** place in this function for you to fill in.

In [ ]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    preds = data.apply(lambda x: classify(tree, x))
    
    # Once you've made the predictions, calculate the classification error and return it
    num_mistakes = (preds != data[target]).sum()
    return float(num_mistakes / len(data))

Now, let's use this function to evaluate the classification error on the test set.

In [ ]:
err = evaluate_classification_error(dt6_model, test_data, target)

# error             accuracy
err, round(err, 2), 1. - err, round(1. - err, 2)

**Quiz Question:** Rounded to 2nd decimal point, what is the classification error of `dt6_model` on the **test_data**?

In [ ]:
round(err, 2)

## Printing out a decision stump

As discussed in the lecture, we can print out a single decision stump (printing out the entire tree is left as an exercise to the curious reader). 

In [ ]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print("(leaf, label: %s)" % tree['prediction'])
        return None
    split_feature, split_value = split_name.split('.')
    print('                       %s' % name)
    print('         |---------------|----------------|')
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print('         |                                |')
    print('         |                                |')
    print('         |                                |')
    print('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))

In [ ]:
print_stump(dt6_model)

**Quiz Question:** What is the feature that is used for the split at the root node?

In [ ]:
dt6_model['splitting_feature']

### Exploring the intermediate left subtree

The tree is a recursive dictionary, so we do have access to all the nodes! We can use
* `dt6_model['left']` to go left
* `dt6_model['right']` to go right

In [ ]:
print_stump(dt6_model['left'], dt6_model['splitting_feature'])

### Exploring the left subtree of the left subtree


In [ ]:
print_stump(dt6_model['left']['left'], dt6_model['left']['splitting_feature'])

**Quiz Question:** What is the path of the **first 3 feature splits** considered along the **left-most** branch of **my_decision_tree**?

In [ ]:
print_stump(dt6_model['left']['left']['left'], dt6_model['left']['left']['splitting_feature'])
# term. 36 months / grade.A / grade.B

**Quiz Question:** What is the path of the **first 3 feature splits** considered along the **right-most** branch of **my_decision_tree**?

In [ ]:
print_stump(dt6_model['right'], dt6_model['splitting_feature'])   # => leaf
# term. 36 months / grade.D